# **Lecture des données**

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation des données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Lecture des données d'apprentissage et de test
dataset = pd.read_csv("../input/housesalesprediction/kc_house_data.csv")

In [ ]:
# Récupération des premières lignes du dataset
dataset.head(10)

In [ ]:
# Récupération des informations sur le dataset
dataset.info

In [ ]:
# Affichage des maisons en fonction de la longitude et de la latitude
dataset.plot(kind="scatter", x="long", y="lat", c="price", cmap="rainbow", s=3, figsize=(12,12))

In [ ]:
# Vue des statistiques sur le dataset
dataset.describe()

# **Interprétation des paramètres**
- id : id de la vente
- date : date de la vente
- price : prix de la vente
- bedrooms : nombre de chambre
- bathrooms : nombre de salle de bain
- sqft_living : surface de la maison
- sqft_lot : surface totale du terrain
- floors : nombre d'étage
- waterfront : vue sur la mer
- view :
- condition : note en fonction de l'age et de la qualité de la construction (1-5)
- grade : note en fonction de la qualité de la construction (1-13)
- sqft_above : surface de la maison - surface du sous-sol
- sqft_basement : surface du sous-sol
- yr_built : année de construction
- yr_renovated : année de rénovation
- zipcode : code postal
- lat : latitude
- long : longitude
- sqft_living15 : surface moyenne des 15 maisons les plus proches
- sqft_lot15 : surface moyenne des 15 terrains (maisons y compris) les plus proches

In [ ]:
plt.scatter(dataset["sqft_living"], dataset["price"])
plt.xlabel("Taille de la propriété")
plt.ylabel("Prix de vente")
plt.show()

On peut remarquer que si la taille de la propriété augmente, le prix augmente

In [ ]:
plot = dataset.groupby("grade")["price"].mean()
plt.scatter(np.unique(dataset["grade"]), plot)
plt.xlabel("Condition de la propriété (de 1 à 13)")
plt.ylabel("prix de vente moyen")
plt.show()

On peut remarquer que si la note (grade) est élevée, le prix augmente 

In [ ]:
plot = dataset.groupby("yr_built")["price"].mean()
plt.scatter(np.unique(dataset["yr_built"]), plot)
plt.xlabel("Année")
plt.ylabel("prix de vente moyen")
plt.show()

On remarque que les maisons construites entre 1940 et 1990 ont un prix moins élevé

In [ ]:
plot = dataset.groupby("yr_built")["price"].count()
plt.scatter(np.unique(dataset["yr_built"]), plot)

In [ ]:
condition = dataset['condition'].value_counts()

print("Nombre de logement par condition : ")
print(condition)

fig, ax = plt.subplots(ncols=2, figsize=(14,5))
sns.countplot(x='condition', data=dataset, ax=ax[0])
sns.boxplot(x='condition', y= 'price',
            data=dataset, ax=ax[1])
plt.show()

# **Recherche de corrélations**

In [ ]:
dataset.columns

In [ ]:
# On retire les colonnes id, lat, long, et code postal ce qui permettra d'éviter des erreurs
houseSales = dataset.drop(['id','date','lat','long','zipcode'], axis=1)

In [ ]:
tabcorr = houseSales.corr()
plt.figure(figsize=(12,12))
sns.heatmap(abs(tabcorr), cmap="coolwarm")

In [ ]:
correlations = tabcorr.price
print(correlations)

On élimine la ligne que l'on va prédire (égale à 1)

In [ ]:
correlations = correlations.drop(['price'], axis=0)

Les corrélations négatives sont aussi importantes et significatives que les positives, on prend donc les valeurs absolues et on trie par ordre décroissant

In [ ]:
print(abs(correlations).sort_values(ascending=False))

> On voit que la corrélation la plus importe est la surface de la maison

# **Régression linéaire multiple**

In [ ]:
continuous_features = ['sqft_living', 'sqft_above', 'sqft_lot', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
discrete_features = ['grade', 'bathrooms', 'view', 'bedrooms', 'waterfront', 'floors', 'yr_renovated', 'yr_built', 'condition']

In [ ]:
# Pour la régression linéaire, on se limite aux maisons de moins de 1M$, et on élimine les features discrètes :
houseSales2 = houseSales[houseSales.price<1000000].drop(discrete_features, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X = houseSales2.drop(['price'], axis=1)
Y = houseSales2.price
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)

In [ ]:
# On utilise la régression linéaire de sklearn
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, Y_train)                     # Apprentissage
Y_pred = lr.predict(X_test)                  # Prédiction sur l'ensemble de test

In [ ]:
# On trace le nuage de points pour comparer la prédiction et les résultats attendus :
plt.figure(figsize=(12,12))
plt.scatter(Y_test, Y_pred)
plt.plot([Y_test.min(),Y_test.max()],[Y_test.min(),Y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
# On peut visualiser la distribution de l'erreur avec seaborn :
sns.distplot(Y_test-Y_pred)

In [ ]:
# On calcule l'erreur sur les moindres carrés :
from sklearn.metrics import mean_squared_error, r2_score

print(np.sqrt(mean_squared_error(Y_test, Y_pred)))

In [ ]:
# On calcule le score R2 (rapport des variances estimée/réelle) :
scoreR2 = r2_score(Y_test, Y_pred)
print(scoreR2)

# **Régression par forêts aléatoires**
Test de la régression avec les forêts aléatoires

In [ ]:
X = houseSales2.drop(['price'], axis=1)
Y = houseSales2.price
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble

randomForest = ensemble.RandomForestRegressor()
randomForest.fit(X_train, Y_train)
Y_randomForest = randomForest.predict(X_test)
print(randomForest.score(X_test,Y_test))

In [ ]:
# On trace le nuage de points pour comparer la prédiction et les résultats attendus :
plt.figure(figsize=(12,12))
plt.scatter(Y_test, Y_pred)
plt.plot([Y_test.min(),Y_test.max()],[Y_test.min(),Y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
sns.distplot(Y_test-Y_randomForest)

In [ ]:
print(np.sqrt(mean_squared_error(Y_test, Y_randomForest)))